In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from keras.models import load_model
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
IMG_SIZE = 100
NUM_FRAME= 10

In [3]:
cnn = load_model('CNN-BiLSTM.h5')
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 10, 100, 100, 64)  1792      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 98, 98, 64)    36928     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 10, 49, 49, 64)    0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 47, 47, 64)    36928     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 10, 23, 23, 64)    0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 21, 21, 64)    36928     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 10, 10, 64)   

In [8]:
def create_dataset():
    dataset = []
    image = []
    limit = 0
    c = 0

    for file in tqdm(os.listdir('./raw/Dataframes/')):
        path = os.path.join('./raw/Dataframes/', file)
        img = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        image.append(np.array(img))
        limit += 1
        c += 1
        if c == NUM_FRAME:
            c = 0
            if limit < 13622:
                dataset.append([image, np.array([0, 1])])
            elif limit >= 13622:
                dataset.append([image, np.array([1, 0])])
            image = []
    
    shuffle(dataset)
    np.save('dataset_raw.npy', dataset)
    return dataset

In [9]:
data = create_dataset()

100%|██████████| 30787/30787 [07:16<00:00, 70.58it/s]
C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [4]:
data = np.load('dataset_raw.npy', allow_pickle=True)

In [11]:
len(data)

3078

In [10]:
X = np.array([i[0] for i in data]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in data])

In [12]:
X = X.astype('float32')/255

In [13]:
cnn.evaluate(x = X, y =Y, batch_size= 5, verbose = 1)

616/616 [==============================] - 25s 41ms/step - loss: 1.4818 - accuracy: 0.5387


[1.481820821762085, 0.5386614799499512]